In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from coordinates import convert_to_utm


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df = pd.read_csv("Label_Data")

In [19]:
df.loc[df["Traffic_Label"] == "No Traffic", "Traffic_Label"] = 0
df.loc[df["Traffic_Label"] == "Low Traffic", "Traffic_Label"] = 1
df.loc[df["Traffic_Label"] == "Moderate", "Traffic_Label"] = 2
df.loc[df["Traffic_Label"] == "Dense", "Traffic_Label"] = 3
df.loc[df["Traffic_Label"] == "Very Dense", "Traffic_Label"] = 4

In [20]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Filter out data for January 2nd
df_filtered = df[df['Timestamp'].dt.date != pd.to_datetime('2019-01-02').date()]

In [22]:
df_filtered.info()
df = df_filtered

<class 'pandas.core.frame.DataFrame'>
Index: 82305 entries, 0 to 109056
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype                    
---  ------         --------------  -----                    
 0   Timestamp      82305 non-null  datetime64[ns, UTC+03:00]
 1   Longitude      82305 non-null  float64                  
 2   Latitude       82305 non-null  float64                  
 3   Traffic_Label  82305 non-null  object                   
dtypes: datetime64[ns, UTC+03:00](1), float64(2), object(1)
memory usage: 3.1+ MB


In [23]:
from sklearn.preprocessing import LabelEncoder 
  
le = LabelEncoder() 
  
df['Timestamp']= le.fit_transform(df['Timestamp'])
df2 = df

In [24]:
convert_time = convert_to_utm(df, src_epsg=4326, dst_epsg=32633,
                        col_lat='Latitude', col_lon='Longitude')

Formal definition string for the old projection: proj=longlat datum=WGS84 no_defs ellps=WGS84 towgs84=0,0,0
Formal definition string for the new projection: proj=utm zone=33 datum=WGS84 units=m no_defs ellps=WGS84 towgs84=0,0,0


c:\Users\ysrmhmt\Desktop\Lectures\Data Mining\Project\Group_2\coordinates.py:16: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2, y2 = pyproj.transform(old_proj, new_proj, x1, y1)


In [25]:
df2.head()

,Timestamp,Longitude,Latitude,Traffic_Label
0,0,3.001412e+06,3.481096e+06,0
1,1,3.001676e+06,3.480843e+06,0
2,2,3.002225e+06,3.480731e+06,0
3,3,3.002212e+06,3.480716e+06,1
4,7,3.002207e+06,3.480856e+06,1


In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = df.drop(["Traffic_Label"], axis=1)
y = df["Traffic_Label"]

X_train, X_test, y_train,y_test = train_test_split(X,y, test_size= 0.3, random_state= 0)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [29]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train,y_train)


SVR()

In [30]:
y_pred = regressor.predict(X_test)

In [31]:
accuracy = regressor.score(X_test,y_test)


In [32]:
accuracy

0.38980967014797885

In [56]:

if(y_pred.all()<2.5):
    y_pred=np.round(y_pred-0.5)
    
else:
    y_pred=np.round(y_pred+0.5)

#Calculationg the error and accuracy

df1=(y_pred-y_test)/y_test
df1=round(df1.mean()*100,2)
print("Error = ",df1,"%") 
a=100-df1
print("Accuracy= ",a,"%")


ZeroDivisionError: float division by zero

In [31]:
accuracy = svc_model.score(X_test,y_test)
print('\n Accuracy:\n', accuracy)

# EVALUATING THE MODEL
y_predict = svc_model.predict(X_test)



 Accuracy:
 -0.06117192507605229


ValueError: Classification metrics can't handle a mix of unknown and continuous targets

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
df['Hour'] = df['Timestamp'].dt.hour
df['Day'] = df['Timestamp'].dt.day
df['Month'] = df['Timestamp'].dt.month

scaler = StandardScaler()
features = ['Hour', 'Day', 'Month', 'Long', 'Lat'] # Add your feature names here
df[features] = scaler.fit_transform(df[features])

X = df[features]
y = df['Traffic_Density_Label'] # Replace with your column name
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

svc_model = SVC()
svc_model.fit(X_train, y_train)
